# Dissertation

## Research into the techniques and methods to achieve state of the art accuracy in flower species identification


In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
###imports###
import numpy as np
import tensorflow as tf
import scipy.io
from scipy import io
import time
import sys
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing import image
from tensorflow.python.framework import graph_util
from tensorflow.keras.callbacks import TensorBoard
from keras.callbacks import TensorBoard
import pickle
import datetime
import time
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import OneHotEncoder
import logging
#import skimage.io
import random
import PIL
from pathlib import Path

In [64]:
print(tf.__version__)

2.5.0


In [65]:
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [68]:
#makes all outputs be in float format rather than exponentials
#np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [69]:
#Set Parameters
IMG_SIZE = 224 #this parameter sets image dimensions as 50*50
DATE = datetime.datetime.now().strftime('%d-%b-%Y')
MODEL_PATH = f'models/{DATE}/'
MODEL_NAME = 'FlowerClassifierTrial.model'.format(int(time.time()))
log_dir=f'logs\\{MODEL_NAME}'
TENSORBOARD = TensorBoard(log_dir)

In [70]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [71]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Load Directory

In [72]:
# ###Directory for MacBook
# mac_URL = '/Volumes/T7/Uni/Github/thesis/flowerDataset/Flower'
# mac_labels = '/Volumes/T7/Uni/Github/thesis/flowerDataset/imagelabels.mat'
# mac_dataSplit = '/Volumes/T7/Uni/Github/thesis/flowerDataset/setid.mat'
# mac_loaded_images = '/Volumes/T7/Uni/loaded_images.npy'
# mac_image_train='/Volumes/T7/Uni/image_train.npy'
# mac_label_train = '/Volumes/T7/Uni/label_train.npy'
# mac_image_test = '/Volumes/T7/Uni/image_test.npy'
# mac_label_test = '/Volumes/T7/Uni/label_test.npy'
# mac_image_val = '/Volumes/T7/Uni/image_val.npy'
# mac_label_val = '/Volumes/T7/Uni/label_val.npy'

# ###Directory for PC
# pc_URL = 'E:/Github/thesis/flowerDataset/Flower'
# pc_labels = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
# pc_dataSplit = 'E:/Github/thesis/flowerDataset/setid.mat'
# pc_loaded_images = 'E:/Dissertation/data/loaded_images.npy'
# pc_image_train = 'E:/Dissertation/data/image_train.npy'
# pc_label_train = 'E:/Dissertation/data/label_train.npy'
# pc_image_test = 'E:/Dissertation/data/image_test.npy'
# pc_label_test = 'E:/Dissertation/data/label_test.npy'
# pc_image_val = 'E:/Dissertation/data/image_val.npy'
# pc_label_val = 'E:/Dissertation/data/label_val.npy'

In [73]:
def get_directories():
    if os.path.exists(mac_URL) and os.path.exists(mac_labels) and os.path.exists(
            mac_dataSplit) and os.path.exists(mac_loaded_images):
        return mac_URL, mac_labels, mac_dataSplit, mac_loaded_images, mac_image_train, mac_label_train, mac_image_test, mac_label_test,mac_image_val, mac_label_val 
    else:
        return pc_URL, pc_labels, pc_dataSplit, pc_loaded_images, pc_image_train, pc_label_train, pc_image_test, pc_label_test, pc_image_val, pc_label_val

In [74]:
#URL_dir, labels_dir, dataSplit_dir, loaded_images_dir, image_train_dir, label_train_dir, image_test_dir, label_test_dir, image_val_dir, label_val_dir = get_directories()

In [75]:
URL_dir = 'E:/Github/thesis/flowerDataset/Flower'
labels_dir = 'E:/Github/thesis/flowerDataset/imagelabels.mat'
dataSplit_dir = 'E:/Github/thesis/flowerDataset/setid.mat'
loaded_images_dir = 'E:/Dissertation/data/loaded_images.npy'
image_train_dir ='E:/Dissertation/data/image_train.npy'
label_train_dir = 'E:/Dissertation/data/label_train.npy'
image_test_dir = 'E:/Dissertation/data/image_test.npy'
label_test_dir = 'E:/Dissertation/data/label_test.npy'
image_val_dir = 'E:/Dissertation/data/image_val.npy'
label_val_dir = 'E:/Dissertation/data/label_val.npy'

In [76]:
print(labels_dir)

E:/Github/thesis/flowerDataset/imagelabels.mat


### Load DataSet and Labels

In [77]:
data_labels = io.loadmat(labels_dir)
data_labels = data_labels.items()
data_labels = list(data_labels)

In [78]:
df = pd.DataFrame(data_labels)

In [79]:
labels = df[1][3][0] #loads the labels that is stored in dataframe format

In [80]:
labels

array([77, 77, 77, ..., 62, 62, 62], dtype=uint8)

In [81]:
len(labels)

8189

In [82]:
UNIQUE_LABELS = np.unique(labels)
UNIQUE_LABELS

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102], dtype=uint8)

In [83]:
#Got loading files code from link below
#https://stackoverflow.com/questions/30230592/loading-all-images-using-imread-from-a-given-folder
#Function gets all the images loaded up
def load_images_from_folder(URL):
    images = []
    for filename in os.listdir(URL):
        #img = cv2.imread(os.path.join(URL,filename))
        img = cv2.resize(cv2.imread(os.path.join(URL,filename), cv2.COLOR_BGR2RGB), (IMG_SIZE, IMG_SIZE))
        img = np.reshape(img,[IMG_SIZE,IMG_SIZE,3])
        if img is not None:
            images.append(img)
    np.save(loaded_images_dir, images)
    return images

In [85]:
#No need to run this cell if image already loaded.
#load_images_from_folder(URL_dir)

In [86]:
#if images are already loaded, just load the file here rather than running function again.
image_data = np.load(loaded_images_dir, allow_pickle=True)

In [87]:
#shows the shape of the image_data, the number of images, the dimensions and number of colour channels
image_data.shape

(8189, 224, 224, 3)

In [88]:
image_data[0].shape

(224, 224, 3)

In [89]:
#combine image with labels
def randomised_dataset(image_data, labels):
    np.random.seed(8)
    indices = np.arange(image_data.shape[0])
    print(indices)
    np.random.shuffle(indices)
    image_data = image_data[indices]
    labels = labels[indices]
    return image_data, labels

In [90]:
#image_data, labels = randomised_dataset(image_data, labels)

In [91]:
print(image_data.shape)

(8189, 224, 224, 3)


In [92]:
# # Label Encoder

mlb = LabelBinarizer()
converted_labels = np.array(mlb.fit_transform(labels))

In [93]:
converted_labels

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [94]:
def convert_label(labels):
    new_labels = []
    for i in range(len(labels)):
        new_labels.insert(i, [labels[i]])
    return np.array(new_labels)

In [95]:
#converted_labels = convert_label(labels)

In [96]:
#converted_labels.shape

In [97]:
len(converted_labels)

8189

In [98]:
converted_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Training and Testing Split

In [99]:
#citation code from https://datascience.stackexchange.com/questions/15135/train-test-validation-set-splitting-in-sklearn
#splitting the image dataset into the ratio for training, validation and testing data
def split_data(image_data, converted_labels):
    
    train_ratio = 0.75
    validation_ratio = 0.15
    test_ratio = 0.10

    image_train, image_test, label_train, label_test = train_test_split(image_data, converted_labels, test_size=1 - train_ratio, random_state=42)

    #the test from previous line which is 25% of dataset is passed into the line below to be 
    #further split into 15% for validation and 10% for testing

    image_val, image_test, label_val, label_test = train_test_split(image_test, label_test, test_size=test_ratio/(test_ratio + validation_ratio),random_state=42) 

    print('image_train',image_train.shape)
    print('label_train',label_train.shape)
    print('image_test',image_test.shape)
    print('label_test',label_test.shape)
    print('image_val', image_val.shape)
    print('label_val', label_val.shape)

    np.save(image_train_dir, image_train)
    np.save(label_train_dir, label_train)
    np.save(image_test_dir, image_test)
    np.save(label_test_dir, label_test)
    np.save(image_val_dir, image_val)
    np.save(label_val_dir, label_val)



In [100]:
split_data(image_data,converted_labels)

image_train (6141, 224, 224, 3)
label_train (6141, 102)
image_test (820, 224, 224, 3)
label_test (820, 102)
image_val (1228, 224, 224, 3)
label_val (1228, 102)


In [101]:
image_train = np.load(image_train_dir, allow_pickle=True)
image_test = np.load(image_test_dir, allow_pickle=True)
label_train = np.load(label_train_dir, allow_pickle=True)
label_test = np.load(label_test_dir, allow_pickle=True)
image_val = np.load(image_val_dir, allow_pickle=True)
label_val = np.load(label_val_dir, allow_pickle=True)

In [102]:
image_test.shape

(820, 224, 224, 3)

In [103]:
label_val[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Feature Scaling

In [104]:
#converts the images to the range of 0 - 1.0 
image_train = image_train/255.0
image_test = image_test/255.0
image_val = image_val/255.0

In [105]:
image_train.shape

(6141, 224, 224, 3)

In [106]:
label_train.shape

(6141, 102)

In [107]:
label_test.shape

(820, 102)

In [108]:
len(image_test)

820

In [109]:
label_train[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [110]:
# number of classes/ flowers
NUM_CLASSES = 102
NUM_CLASSES

102

In [111]:
label_train.shape

(6141, 102)

In [112]:
label_train[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0,

# Custom CNN model

In [162]:
# name of model
name_model = 'test-{}'.format(int(time.time()))

In [163]:
batch_size = 8
epochs = 100
verbose= 2
act = 'relu'

In [164]:
def create_cnn_model(act):
    # tf.reset_default_graph()
    model = Sequential()
    input_shape = (IMG_SIZE, IMG_SIZE, 3)
    
    # INPUT LAYER
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation(act))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    # HIDDEN LAYER 1
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))

    # HIDDEN LAYER 2
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))
    
    # HIDDEN LAYER 3
    model.add(Conv2D(128, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))
    
    # HIDDEN LAYER 4
    model.add(Conv2D(256, (3, 3)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    # model.add(Dropout(0.25))

    # Fully Connected
    model.add(Flatten()) # converts the 3D feature maps to 1D feature vectors
    model.add(Dense(256))
    model.add(Activation(act))
    model.add(Dropout(0.25)) # reduces overfitting

    # OUTPUT LAYER
    model.add(Dense(NUM_CLASSES))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [165]:
model = create_cnn_model(act)

In [166]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
activation_42 (Activation)   (None, 222, 222, 32)      0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 220, 220, 32)      9248      
_________________________________________________________________
activation_43 (Activation)   (None, 220, 220, 32)      0         
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 110, 110, 32)      0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 110, 110, 32)      128       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 108, 108, 64)     

In [167]:
#history = model.fit(image_train, label_train, batch_size=64, epochs=100, validation_data=(image_val, label_val), verbose=1, callbacks = [tensorboard])

In [168]:
#saving model
#model.save(f'{MODEL_PATH}')

In [169]:
#loading model
#model =  tf.keras.models.load_model(f'{MODEL_PATH}')
#model = tf.keras.models.load_model(f'E:/GoogleSync/Masters/Dissertation/models/24-Jun-2021') 

# Evaluation

In [170]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [172]:
EXPERIMENT_NAME = "custom-model-tests"
RUN_NAME = "eight_batch_default_cnn_colour"

mlflow.set_experiment(EXPERIMENT_NAME)
mlflow.set_tracking_uri('file:///E:/GoogleSync/Masters/Dissertation/MLflow/mlruns')

with mlflow.start_run(run_name=RUN_NAME) as run:
        
        mlflow.tensorflow.autolog()
        
        history = model.fit(image_train, label_train, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(image_val, label_val), callbacks=[tensorboard_callback])

        score = model.evaluate(image_test, label_test, batch_size=batch_size, verbose = verbose)
        
        
        
    
        
        mlflow.log_param("activation function", act)
        mlflow.log_metric("test loss", score[0])
        mlflow.log_metric("test accuracy", score[1])
        mlflow.log_param("epochs", epochs)
        mlflow.log_param("batch_size", batch_size)
        
        
        
        #tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        #mlflow.keras.log_model(model, "standardCNN")        

2021/08/08 15:13:08 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


Epoch 1/100


768/768 - 15s - loss: 4.9109 - accuracy: 0.0259 - val_loss: 5.8558 - val_accuracy: 0.0399
Epoch 2/100
768/768 - 10s - loss: 4.5779 - accuracy: 0.0321 - val_loss: 4.5908 - val_accuracy: 0.0318
Epoch 3/100
768/768 - 10s - loss: 4.5418 - accuracy: 0.0353 - val_loss: 4.5874 - val_accuracy: 0.0301
Epoch 4/100
768/768 - 10s - loss: 4.5126 - accuracy: 0.0365 - val_loss: 7.5044 - val_accuracy: 0.0358
Epoch 5/100
768/768 - 10s - loss: 4.5616 - accuracy: 0.0319 - val_loss: 4.6321 - val_accuracy: 0.0334
Epoch 6/100
768/768 - 10s - loss: 4.5153 - accuracy: 0.0331 - val_loss: 4.5235 - val_accuracy: 0.0293
Epoch 7/100
768/768 - 10s - loss: 4.5063 - accuracy: 0.0314 - val_loss: 4.5218 - val_accuracy: 0.0293
Epoch 8/100
768/768 - 10s - loss: 4.4938 - accuracy: 0.0321 - val_loss: 4.5215 - val_accuracy: 0.0375
Epoch 9/100
768/768 - 10s - loss: 4.4932 - accuracy: 0.0305 - val_loss: 4.5214 - val_accuracy: 0.0366
Epoch 10/100
768/768 - 10s - loss: 4.4865 - accuracy: 0.0337 - val_loss: 4.5252 - val_accuracy

Epoch 81/100
768/768 - 12s - loss: 4.4988 - accuracy: 0.0324 - val_loss: 4.5228 - val_accuracy: 0.0366
Epoch 82/100
768/768 - 12s - loss: 4.4898 - accuracy: 0.0322 - val_loss: 4.5230 - val_accuracy: 0.0366
Epoch 83/100
768/768 - 11s - loss: 4.4911 - accuracy: 0.0322 - val_loss: 4.5225 - val_accuracy: 0.0366
Epoch 84/100
768/768 - 11s - loss: 4.4862 - accuracy: 0.0329 - val_loss: 4.5231 - val_accuracy: 0.0366
Epoch 85/100
768/768 - 11s - loss: 4.4898 - accuracy: 0.0322 - val_loss: 4.5226 - val_accuracy: 0.0366
Epoch 86/100
768/768 - 11s - loss: 4.4898 - accuracy: 0.0309 - val_loss: 4.5233 - val_accuracy: 0.0366
Epoch 87/100
768/768 - 12s - loss: 4.4876 - accuracy: 0.0327 - val_loss: 4.5233 - val_accuracy: 0.0366
Epoch 88/100
768/768 - 11s - loss: 4.4883 - accuracy: 0.0318 - val_loss: 4.5231 - val_accuracy: 0.0366
Epoch 89/100
768/768 - 11s - loss: 4.4936 - accuracy: 0.0314 - val_loss: 4.5255 - val_accuracy: 0.0366
Epoch 90/100
768/768 - 11s - loss: 4.4921 - accuracy: 0.0331 - val_loss: 

INFO:tensorflow:Assets written to: C:\Users\yisi9\AppData\Local\Temp\tmpjet1vn5t\model\data\model\assets


103/103 - 1s - loss: 4.5118 - accuracy: 0.0293


In [173]:
#function that returns loss value & metrics values
model.evaluate(image_test, label_test)

26/26 [==============================] - 0s 15ms/step - loss: 4.5118 - accuracy: 0.0293


[4.511775493621826, 0.02926829271018505]

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=logs

In [ ]:
def prediction_accuracy(image_test, model):
    #going through each of the test images
    count = 0
    for i in range(len(image_test)): #going through all test images
        img = image_test[i] #load in the image
        
        img = img.reshape(-1, 100, 100, 3) #get it in the right shape for model.predict
        
        prediction = model.predict(img).flatten() #using model predict to get what it thinks is the answer
        
        # prediction.astype(np.int) #converts predictions to integers
        
        pred_index = np.argmax(prediction) # grabs the INDEX of the best prediction
        
        max_pred = max(prediction) # grabs the SCORE of best prediction
        # print(prediction)
        # print(pred_index)
        
        test_label_value = label_test[i][pred_index] # gets the actual test label at the same index
        # print(test_label_value)
        if test_label_value == 1:
            #print("CORRECT PREDICTION")
            count += 1
            
#         # FIND THE INDEX WHERE THE VALUE = 1
#         for j in range(len(predicted_label)):
#             if predicted_label[j] == 1 : #find the point when it finds 1 
#                 if label_test[i][j] == 1: #check if at this exact point is also 1  whic means correct prediction
#                     count+=1 #increase number of correct count by 1
#                     print(count)
#                     break
    print(f"num correct - {count}")
    accuracy = (count/len(label_test))*100
    per_symbol = '%'
    print(f'accuracy is {accuracy} {per_symbol}')        

In [ ]:
image_test.shape

In [ ]:
prediction_accuracy(image_test, model) 